In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from rt1_dataset_wrapper import RT1Dataset

In [3]:
dataset = RT1Dataset("/scratch/gsk6me/WORLDMODELS/datasets/rt-1-public-release")

# import tensorflow_datasets as tfds

# builder = tfds.builder_from_directory('/scratch/gsk6me/WORLDMODELS/datasets/rt-1-data-release')
# dataset_tfds = builder.as_data_source()


In [4]:
# Visualize 
from diffusers import AutoencoderKLTemporalDecoder
import torch
from diffusers import StableVideoDiffusionPipeline

model_path = "stabilityai/stable-video-diffusion-img2vid-xt"
pipeline = StableVideoDiffusionPipeline.from_pretrained(model_path)
pipeline = pipeline.to(device='cuda')
vae = pipeline.vae


/home/gsk6me/miniconda3/envs/py310/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/gsk6me/miniconda3/envs/py310/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
2024-03-01 00:16:17.996753: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/gsk6me/miniconda3/envs/py310/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: Use

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

In [22]:
import numpy as np
import os
import subprocess
from IPython.display import Video, display
import cv2

def render_video(tensor_sequence, index):
    # Convert the PyTorch tensors to numpy arrays
    frames = [(tensor.permute(1, 2, 0).cpu().numpy()[:, :, ::-1] * 255).astype(np.uint8) for tensor in tensor_sequence]

    # Define the video writer
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('_generated.mp4', fourcc, 30.0, (width, height))

    # Write each frame to the video
    for frame in frames:
        out.write(frame.astype(np.uint8))

    # Release the video writer
    out.release()

    with open(os.devnull, 'wb') as devnull:
        subprocess.check_call(['ffmpeg', '-y', '-i', '_generated.mp4', f'generated_{index}.mp4'], stdout=devnull, stderr=devnull)

    os.system('rm _generated.mp4')
    
    return Video(f'generated_{index}.mp4')

In [23]:
## Visualize how good the autoencoder is ##

for index in [18, 180, 1800, 18000]:
    text_description, video = dataset[index]
    
    print("Original Video")
    display(render_video(video, 'original_' + str(index)))
    
    # Now, let's see what it looks like if we pass it in and out of the VAE.
    with torch.no_grad():
        latents = torch.stack([
            vae.encode(
                pipeline.image_processor.preprocess(
                    image.to(device='cuda', dtype=torch.float),
                    height=256, width=320,
                )
            ).latent_dist.mode()[0]
            for image in video
        ])
        
        # Decode latents in chunks of 8
    
        returned = []
        i = 0
        while i < len(latents):
            frames = latents[i:i + 8]
            returned.append(vae.decode(frames, num_frames=len(frames)).sample)
            i += 8
    
        returned = (1 + torch.cat(returned)) / 2
        returned = torch.clamp(returned, 0, 1)
    
    print("Autoencoder Video")
    display(render_video(returned, 'autoencoder_' + str(index)))

Original Video


Autoencoder Video


Original Video


Autoencoder Video


Original Video


Autoencoder Video


Original Video


Autoencoder Video
